In [ ]:
pip install unidecode --quiet

from unidecode import unidecode

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from scipy import stats
import re
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN

from sklearn.preprocessing import LabelEncoder
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!kaggle datasets download -d phucvr/lazada-crawl --quiet
!unzip 'lazada-crawl.zip'

# !gdown 1kra_R95S4-metn36lJ5wMn4f6QscnTLr --quiet
data_full = pd.read_excel('/content/filtered_data_product.xlsx')

In [ ]:
data_product = data_full.copy()

# Bước 1: Quan sát sơ bộ dữ liệu
print("Số lượng bản ghi:", len(data_product))
print("Số lượng thuộc tính:", len(data_product.columns))
print("Tên các thuộc tính:", data_product.columns.tolist())
print("Kiểu dữ liệu của các thuộc tính:")
print(data_product.dtypes)

In [ ]:
# Bước 2: Kiểm tra dữ liệu bị thiếu của product
print("Số lượng giá trị bị thiếu trong mỗi thuộc tính:")
print(data_product.isnull().sum())

print("Tỷ lệ phần trăm giá trị bị thiếu trong mỗi thuộc tính:")
print(data_product.isnull().sum() / len(data_product) * 100)
data_product = data_product.drop('options', axis = 1)

In [ ]:
# Check if itemId is unique, dtype is int64, and itemId > 0
if not data_product['itemId'].is_unique or data_product['itemId'].dtype != 'int64' or (data_product['itemId'] <= 0).any():
    print("Cảnh báo: itemId không hợp lệ!")

# Check if name is unique, dtype is string, and has Vietnamese characters
if data_product['name'].isnull().any() or not data_product['name'].apply(lambda x: re.match("^[a-zA-Z0-9ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂẾưăạảấầẩẫậắằẳẵặẹẻẽềềểếỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ\s]+$", str(x))).all():
    print("Cảnh báo: Tên sản phẩm không hợp lệ!")

# Check if priceShow, price, originalPrice > 0 and originalPrice >= priceShow
if ((data_product[['priceShow', 'originalPrice']] < 0).any(axis=1)).any() or (data_product['originalPrice'] < data_product['priceShow']).any():
    print("Cảnh báo: Giá không hợp lệ!")

# Check if discount > 0 and discount <= 100
if ((data_product['discount'] < 0) | (data_product['discount'] > 100)).any():
    print("Cảnh báo: Giảm giá không hợp lệ!")

# Check if ratingScore > 0 and ratingScore <= 5
if ((data_product['ratingScore'] < 0) | (data_product['ratingScore'] > 5)).any():
    print("Cảnh báo: Điểm đánh giá không hợp lệ!")

# Kiểm tra review là số nguyên không âm
if data_product['review'].dtype != 'int64' or (data_product['review'] < 0).any():
    print("Cảnh báo: Số lượng đánh giá không hợp lệ!")

# Kiểm tra location, sellerName, brandName không rỗng và không chứa ký tự đặc biệt
for col in ['location', 'sellerName', 'brandName']:
    if data_product[col].isnull().any() or not data_product[col].apply(lambda x: re.match("^[a-zA-Z0-9ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂẾưăạảấầẩẫậắằẳẵặẹẻẽềềểếỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ\s]+$", str(x))).all():
        print(f"Cảnh báo: {col} không hợp lệ!")

# Kiểm tra category thuộc tập giá trị xác định
valid_categories = ['Điện thoại di động', 'Máy tính bảng']
if not data_product['category'].isin(valid_categories).all():
    print("Cảnh báo: Danh mục sản phẩm không hợp lệ!")

# Kiểm tra itemSoldCntShow là số nguyên không âm
if data_product['itemSoldCntShow'].dtype != 'int64' or (data_product['itemSoldCntShow'] < 0).any():
    print("Cảnh báo: Số lượng sản phẩm đã bán không hợp lệ!")

In [ ]:
def correlation_analysis(df, target_variable=None, corr_threshold=0.8):
    # Chọn chỉ các cột số
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df_numeric = df[numeric_columns]

    # 1. Tính toán ma trận hệ số tương quan
    corr_matrix = df_numeric.corr()

    # 2. Visualize ma trận hệ số tương quan
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, fmt='.2f')
    plt.title('Ma trận hệ số tương quan')
    plt.tight_layout()
    plt.show()

    # 3. Xác định các cặp biến có tương quan cao
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > corr_threshold:
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

    print(f"Các cặp biến có tương quan cao (|r| > {corr_threshold}):")
    for var1, var2, corr in high_corr_pairs:
        print(f"{var1} - {var2}: {corr:.2f}")

    # 4. Kiểm tra tương quan với biến mục tiêu (nếu có)
    if target_variable and target_variable in df_numeric.columns:
        target_corr = corr_matrix[target_variable].sort_values(ascending=False)
        print(f"\nTương quan với biến mục tiêu ({target_variable}):")
        print(target_corr)

        # Visualize tương quan với biến mục tiêu
        plt.figure(figsize=(10, 6))
        target_corr.drop(target_variable).plot(kind='bar')
        plt.title(f'Tương quan với biến mục tiêu ({target_variable})')
        plt.xlabel('Biến')
        plt.ylabel('Hệ số tương quan')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

    # 5. Phân tích thêm: Visualize scatter plot cho các cặp biến quan trọng
    if high_corr_pairs:
        n_plots = min(len(high_corr_pairs), 3)  # Giới hạn số lượng plots để tránh quá tải
        fig, axes = plt.subplots(n_plots, 1, figsize=(10, 5*n_plots))
        axes = axes.flatten() if n_plots > 1 else [axes]
        for i, (var1, var2, _) in enumerate(high_corr_pairs[:n_plots]):
            sns.scatterplot(data=df_numeric, x=var1, y=var2, ax=axes[i])
            axes[i].set_title(f'Scatter plot: {var1} vs {var2}')
        plt.tight_layout()
        plt.show()

    return corr_matrix, high_corr_pairs

# Sử dụng hàm với dữ liệu đã làm sạch
corr_matrix, high_corr_pairs = correlation_analysis(df_cleaned, target_variable='price', corr_threshold=0.7)

# 6. Phân tích phân phối của các biến quan trọng
important_vars = list(set([var for pair in high_corr_pairs for var in pair[:2]]))
if 'price' in df_cleaned.columns and 'price' not in important_vars:
    important_vars.append('price')

fig, axes = plt.subplots(len(important_vars), 1, figsize=(10, 5*len(important_vars)))
axes = axes.flatten() if len(important_vars) > 1 else [axes]
for i, var in enumerate(important_vars):
    sns.histplot(df_cleaned[var], kde=True, ax=axes[i])
    axes[i].set_title(f'Phân phối của {var}')
plt.tight_layout()
plt.show()

# 7. Tính toán và in ra các thống kê mô tả cho các biến quan trọng
print("\nThống kê mô tả cho các biến quan trọng:")
print(df_cleaned[important_vars].describe())

# 8. In ra thông tin về các cột không phải số
non_numeric_columns = df_cleaned.select_dtypes(exclude=[np.number]).columns
print("\nCác cột không phải số:")
print(non_numeric_columns.tolist())
print("\nMẫu dữ liệu của các cột không phải số:")
print(df_cleaned[non_numeric_columns].head())